In [1]:
# check conda connected
import numpy as np 
import pandas as pd 

print(np.apply_along_axis(lambda row: sum(row)**2, 0, np.arange(9,9,9)))

print(9)


#Check torch install and running cudas
#!pip3 install torch
import torch

print("is gpu?:",
torch.cuda.is_available(),
torch.cuda.device_count())

#WARN: Can't ssh into google colab. Going to have to do locally

0
9
is gpu?: False 0


In [5]:
import torch
torch.ops.load_library("warp_perspective/build/libwarp_perspective.so")
print(torch.ops.my_ops.warp_perspective)
print(torch.ops.my_ops.warp_perspective(torch.randn(32, 32), torch.rand(3, 3)))

def compute(x, y, z):
    return x.matmul(y) + torch.relu(z)
  
inputs = [torch.randn(4, 8), torch.randn(8, 5), torch.randn(4, 5)]
trace = torch.jit.trace(compute, inputs)
print(trace.graph)

my_ops.warp_perspective
tensor([[ 0.0000,  0.4301,  0.0026, -0.0019, -0.0011, -0.0022, -0.0015, -0.0007],
        [ 0.0000, -0.0401, -0.0380, -0.0097,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000]])
graph(%x : Float(4, 8, strides=[8, 1], requires_grad=0, device=cpu),
      %y : Float(8, 5, strides=[5, 1], requires_grad=0, device=cpu),
      %z : Float(4, 5, strides=[5, 1], requires_grad=0, device=cpu)):
  %3 : Float(4, 5, strides=[5, 1], requires_grad=0, device=cpu) = aten::matmul(%x, %y) # /tmp/ipykernel_39935/2

In [7]:
def compute(x, y, z):
    x = torch.ops.my_ops.warp_perspective(x, torch.eye(3))
    return x.matmul(y) + torch.relu(z)
  
inputs = [torch.randn(4, 8), torch.randn(8, 5), torch.randn(8, 5)]
trace = torch.jit.trace(compute, inputs)
print(trace.graph)

@torch.jit.script
def compute(x, y):
  if bool(x[0][0] == 42):
      z = 5
  else:
      z = 10
  return x.matmul(y) + z
print(compute.graph)

graph(%x.1 : Float(4, 8, strides=[8, 1], requires_grad=0, device=cpu),
      %y : Float(8, 5, strides=[5, 1], requires_grad=0, device=cpu),
      %z : Float(8, 5, strides=[5, 1], requires_grad=0, device=cpu)):
  %3 : int = prim::Constant[value=3]() # /tmp/ipykernel_39935/2329074975.py:2:0
  %4 : int = prim::Constant[value=6]() # /tmp/ipykernel_39935/2329074975.py:2:0
  %5 : NoneType = prim::Constant()
  %6 : Device = prim::Constant[value="cpu"]() # /tmp/ipykernel_39935/2329074975.py:2:0
  %7 : bool = prim::Constant[value=0]() # /tmp/ipykernel_39935/2329074975.py:2:0
  %8 : Float(3, 3, strides=[3, 1], requires_grad=0, device=cpu) = aten::eye(%3, %4, %5, %6, %7) # /tmp/ipykernel_39935/2329074975.py:2:0
  %x : Float(8, 8, strides=[8, 1], requires_grad=0, device=cpu) = my_ops::warp_perspective(%x.1, %8) # /root/local/miniconda/lib/python3.7/site-packages/torch/_ops.py:143:0
  %10 : Float(8, 5, strides=[5, 1], requires_grad=0, device=cpu) = aten::matmul(%x, %y) # /tmp/ipykernel_39935/232907

In [8]:
@torch.jit.script
def compute(x, y):
  if bool(x[0] == 42):
      z = 5
  else:
      z = 10
  x = torch.ops.my_ops.warp_perspective(x, torch.eye(3))
  return x.matmul(y) + z

print(compute.graph)

graph(%x.1 : Tensor,
      %y.1 : Tensor):
  %29 : int = prim::Constant[value=1]()
  %19 : NoneType = prim::Constant()
  %3 : int = prim::Constant[value=0]() # /tmp/ipykernel_39935/1391356468.py:3:12
  %6 : int = prim::Constant[value=42]() # /tmp/ipykernel_39935/1391356468.py:3:18
  %11 : int = prim::Constant[value=5]() # /tmp/ipykernel_39935/1391356468.py:4:10
  %12 : int = prim::Constant[value=10]() # /tmp/ipykernel_39935/1391356468.py:6:10
  %18 : int = prim::Constant[value=3]() # /tmp/ipykernel_39935/1391356468.py:7:53
  %5 : Tensor = aten::select(%x.1, %3, %3) # /tmp/ipykernel_39935/1391356468.py:3:10
  %7 : Tensor = aten::eq(%5, %6) # /tmp/ipykernel_39935/1391356468.py:3:10
  %9 : bool = aten::Bool(%7) # /tmp/ipykernel_39935/1391356468.py:3:5
  %z : int = prim::If(%9) # /tmp/ipykernel_39935/1391356468.py:3:2
    block0():
      -> (%11)
    block1():
      -> (%12)
  %23 : Tensor = aten::eye(%18, %19, %19, %19, %19) # /tmp/ipykernel_39935/1391356468.py:7:43
  %x.7 : Tensor = my_o